In [239]:
from dotenv import load_dotenv
import requests
import json
import os
from sqlalchemy import create_engine
import pandas as pd
import time

## Consuming API

In [240]:
"""
    ...
"""
load_dotenv()
api_token = os.getenv("API_Token")

"""
    /v4/competitions/{id}/teams
    List all teams for a particular competition.
"""
headers = {'X-Auth-Token': api_token}


# GET ALL COMPETITIONS --+ 10
uri = "https://api.football-data.org/v4/competitions"
arr_competition_id = []
response = requests.get(uri,headers=headers)
if response.status_code == 200:   
    with open('./data_json/competitions.json', 'w') as f:
        json.dump(response.json(), f, ensure_ascii=False, indent=4)
    for competition in response.json()['competitions']:
        arr_competition_id.append(competition['id'])
else:
    print('Error')

In [244]:
print(arr_competition_id)

[2013, 2016, 2021, 2001, 2018, 2015, 2002, 2019, 2003, 2017, 2152, 2014, 2000]


In [245]:
json_competition_teams = []

for competition_id in arr_competition_id: 
    uri_competition_teams = f"http://api.football-data.org/v4/competitions/{competition_id}/teams"
    response = requests.get(uri_competition_teams,headers=headers)
    if response.status_code == 429:
        print('API Status ... ... [LIMIT ERROR]')
        time.sleep(60)
        print('API Status ... ... [CHECKING]')
        response = requests.get(uri_competition_teams,headers=headers)
        print('API Status ... ... [OK]')

    if response.status_code == 200:
        json_competition_teams.append(response.json())
    else:
        print('Error' + str(response.status_code))

with open('./data_json/competition_teams.json', 'w') as f:
    json.dump(json_competition_teams, f, ensure_ascii=False, indent=4)


API Status ... ... [LIMIT]
API Status ... ... [CHECKING]
API Status ... ... [OK]


In [ ]:
"""
# GET ALL TEAMS --+ 50
uri = "https://api.football-data.org/v4/teams"
response = requests.get(uri,headers=headers)
if response.status_code == 200:
    print(len(response.json()['teams'])) 
    #for match in response.json()['teams']:
    #    print(match)
else:
    print('Error')
"""

In [ ]:
# https://docs.football-data.org/general/v4/errors.html#_429_too_many_requests

## Inserting data into SQLITE

In [246]:
engine = create_engine('sqlite:///./data/db_bcg_project.db') 

In [247]:
data_competitions = None
with open('./data_json/competitions.json', 'r') as file:
    data_competitions = json.loads(file.read())

pd_series_competitions = pd.Series(data_competitions['competitions'])

df_competitions = pd_series_competitions.to_frame()
df_competitions = pd.json_normalize(df_competitions[0])
df_competitions = df_competitions[['id','name']]

df_competitions.head(n=0).to_sql(name='dim_competitions', con=engine, index=False, if_exists='replace')
df_competitions.to_sql(name='dim_competitions', con=engine, index=False, if_exists='append')


13

In [248]:
data_competition_teams = None
with open('./data_json/competition_teams.json', 'r') as file:
    data_competition_teams = json.loads(file.read())

pd_serie_competition_teams = pd.Series(data_competition_teams)

df_competition_teams = pd_serie_competition_teams.to_frame()
df_competition_teams = pd.json_normalize(df_competition_teams[0],'teams',[['competition','id'],['competition','name']])
df_competition_teams = df_competition_teams.rename(columns={'competition.id':'fact_competitions','competition.name':'competition_name','id':'team_id'})

df_competition_teams = df_competition_teams[['team_id','name','shortName','tla','fact_competitions','competition_name']]
df_competition_teams_fact = df_competition_teams[['fact_competitions','team_id']]

df_teams_dim = df_competition_teams[['team_id','name']]
df_teams_dim = df_teams_dim.rename(columns={'team_id':'id'})
df_teams_dim = df_teams_dim.drop_duplicates()


In [249]:
# fact
df_competition_teams_fact.head(n=0).to_sql(name='fact_competitions', con=engine, index=False, if_exists='replace')
df_competition_teams_fact.to_sql(name='fact_competitions', con=engine, index=False, if_exists='append')


311

In [250]:
# dimension
df_teams_dim.head(n=0).to_sql(name='dim_teams', con=engine, index=False, if_exists='replace')
df_teams_dim.to_sql(name='dim_teams', con=engine, index=False, if_exists='append')

264

## Datamart

In [251]:
df_export = pd.read_sql('''
    with dim_teams_cte as (
        select * from dim_teams
    ),
    dim_competitions_cte as (
        select * from dim_competitions
    ),
    fact_competitions_cte as (
        select * from fact_competitions
    ),
    final as (
        select  
            dim_competitions_cte.name,  
            count(dim_teams_cte.id) nm_teams
        from fact_competitions_cte
        left join dim_competitions_cte on dim_competitions_cte.id = fact_competitions_cte.fact_competitions
        left join dim_teams_cte on dim_teams_cte.id = fact_competitions_cte.team_id
        group by dim_competitions_cte.name
        order by nm_teams desc
    )
    select * from final
''',engine)

In [252]:
df_export

,name,nm_teams
0,Copa Libertadores,47
1,UEFA Champions League,32
2,FIFA World Cup,32
3,European Championship,24
4,Championship,24
5,Serie A,20
6,Primera Division,20
7,Premier League,20
8,Campeonato Brasileiro Série A,20
9,Primeira Liga,18


In [225]:
df_export.to_csv('./outputs/data.csv', index=False)  

## Commands

```bash
conda env create -f environment.yaml
conda activate bcg_project
conda deactivate

sudo apt install sqlite3
sqlite3 db_name.db
```